In [23]:
import pandas as pd
from pyvi import ViTokenizer
import re
import nltk

# 1. Import data

In [11]:
filename = "train_nor_811.xlsx"
data = pd.read_excel(filename, engine = "openpyxl")

In [12]:
data.head()

,Unnamed: 0,Emotion,Sentence
0,188,Other,cho mình xin bài nhạc tên là gì với ạ
1,166,Disgust,cho đáng đời con quỷ . về nhà lôi con nhà mày ...
2,1345,Disgust,lo học đi . yêu đương lol gì hay lại thích học...
3,316,Enjoyment,uớc gì sau này về già vẫn có thể như cụ này :))
4,1225,Enjoyment,mỗi lần có video của con là cứ coi đi coi lại ...


In [13]:
data.drop(columns = {"Unnamed: 0"}, axis = 1, inplace = True)
data.head()

,Emotion,Sentence
0,Other,cho mình xin bài nhạc tên là gì với ạ
1,Disgust,cho đáng đời con quỷ . về nhà lôi con nhà mày ...
2,Disgust,lo học đi . yêu đương lol gì hay lại thích học...
3,Enjoyment,uớc gì sau này về già vẫn có thể như cụ này :))
4,Enjoyment,mỗi lần có video của con là cứ coi đi coi lại ...


In [14]:
from sklearn.preprocessing import LabelEncoder
data["emotion_encode"] = data["Emotion"]
encoder = LabelEncoder()
data.emotion_encode = encoder.fit_transform(data.Emotion)
data.head()

,Emotion,Sentence,emotion_encode
0,Other,cho mình xin bài nhạc tên là gì với ạ,4
1,Disgust,cho đáng đời con quỷ . về nhà lôi con nhà mày ...,1
2,Disgust,lo học đi . yêu đương lol gì hay lại thích học...,1
3,Enjoyment,uớc gì sau này về già vẫn có thể như cụ này :)),2
4,Enjoyment,mỗi lần có video của con là cứ coi đi coi lại ...,2


# 2. Data visualization

# 3. Data preprocessing

In [80]:
clean_sentences = []
for i in range(len(data)):
    clean_sentences.append(ViTokenizer.tokenize(data.Sentence[i]))

In [81]:
import json
f = open("sensitive_words.json", encoding="utf-8")
sensitive_words = json.load(f)
f.close()

In [82]:
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

def normalize_sentences(sentences):
    punc_lst = {'.', ',', '...', '-', '“', '”', ':', '(', ')', '"', '!', '&', ';', '?', '*', ']', '>', '…', '’',"``","''"}
    clean_sentences = []
    
    # remove punctuation and lowercase
    for sent in sentences:
        
        # remove emojis
        sent = deEmojify(sent)
        
        sent = nltk.word_tokenize(sent)
        temp = []
        for word in sent:
            word = word.lower()
            if (word in punc_lst or word in sensitive_words):
                continue
            elif(word.isdigit()):
                temp.append("<NUM>")
            else:
                temp.append(word)
        # remove whitespace
        sent = ' '.join(temp)
        
        clean_sentences.append(sent)
        
    return clean_sentences

In [83]:
clean_sentences = normalize_sentences(clean_sentences)

# 4. Model architecture

In [100]:
from tensorflow.keras.layers import Flatten
from tensorflow.keras import layers , activations , models , preprocessing , utils
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import tensorflow as tf

In [93]:
MAX_LEN = 100

tokenizer = Tokenizer(lower = True, split = ' ')
tokenizer.fit_on_texts(clean_sentences)

X = tokenizer.texts_to_sequences(clean_sentences)

X = pad_sequences(X, MAX_LEN, padding='post', truncating='post')

print(X.shape)

(5548, 100)


In [96]:
vocab_size = len(tokenizer.word_index) + 1

In [99]:
embedding_dim = 128

inputs = layers.Input(shape=( MAX_LEN , ))
embedding = layers.Embedding(vocab_size, embedding_dim, input_length=MAX_LEN)(inputs)

cnn1=layers.Conv1D(filters=100, kernel_size=1, activation='relu')(embedding)
cnn1 = layers.MaxPooling1D(pool_size=2)(cnn1)
cnn1 = Flatten()(cnn1)

cnn2=layers.Conv1D(filters=100, kernel_size=2, activation='relu')(embedding)
cnn2 = layers.MaxPooling1D(pool_size=2)(cnn2)
cnn2 = Flatten()(cnn2)

outputs = layers.Concatenate()([cnn1,cnn2])

outputs = layers.Dense(28, activation='tanh')(outputs)
outputs = layers.Dense(14, activation='tanh')(outputs)
outputs = layers.Dense(7, activation='softmax')(outputs)
model=models.Model(inputs,outputs)
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 100, 128)     521344      input_1[0][0]                    
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 100, 100)     12900       embedding[0][0]                  
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 99, 100)      25700       embedding[0][0]                  
_______________________________________________________________________________________

In [101]:
from  sklearn.model_selection import train_test_split
Y = tf.keras.utils.to_categorical(data.emotion_encode)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.25, random_state = 36)

In [104]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)
history = model.fit(X_train, y_train,
                    epochs=10,
                    callbacks = [es],
                    validation_data=(X_test, y_test),
                    batch_size=20)

Epoch 1/10
209/209 [==============================] - 7s 33ms/step - loss: 0.0044 - accuracy: 0.9962 - val_loss: 0.5978 - val_accuracy: 0.4629
Epoch 2/10
209/209 [==============================] - 7s 34ms/step - loss: 0.0129 - accuracy: 0.9873 - val_loss: 0.6650 - val_accuracy: 0.4283
Epoch 00002: early stopping


In [112]:
from keras.models import Sequential
from keras.layers import Embedding, SpatialDropout1D, LSTM, Dense

model2 = Sequential()
model2.add(Embedding(vocab_size, embedding_dim, input_length=MAX_LEN))
model2.add(SpatialDropout1D(0.2))
model2.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model2.add(Dense(7, activation='softmax'))
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [114]:
history2 = model.fit(X_train, y_train,
                    epochs=10,
                    validation_data=(X_test, y_test),
                    batch_size=64)

Epoch 1/10
66/66 [==============================] - 5s 75ms/step - loss: 0.0017 - accuracy: 0.9990 - val_loss: 0.6439 - val_accuracy: 0.4499
Epoch 2/10
66/66 [==============================] - 5s 70ms/step - loss: 0.0015 - accuracy: 0.9990 - val_loss: 0.6502 - val_accuracy: 0.4499
Epoch 3/10
66/66 [==============================] - 5s 70ms/step - loss: 0.0014 - accuracy: 0.9990 - val_loss: 0.6548 - val_accuracy: 0.4499
Epoch 4/10
66/66 [==============================] - 5s 79ms/step - loss: 0.0013 - accuracy: 0.9990 - val_loss: 0.6588 - val_accuracy: 0.4492
Epoch 5/10
66/66 [==============================] - 5s 75ms/step - loss: 0.0013 - accuracy: 0.9986 - val_loss: 0.6625 - val_accuracy: 0.4521
Epoch 6/10
66/66 [==============================] - 5s 72ms/step - loss: 0.0012 - accuracy: 0.9990 - val_loss: 0.6662 - val_accuracy: 0.4513
Epoch 7/10
66/66 [==============================] - 5s 74ms/step - loss: 0.0012 - accuracy: 0.9988 - val_loss: 0.6694 - val_accuracy: 0.4521
Epoch 8/10
66